# Data Cleaning Procedures

In [ ]:
# Constraint 1: Data Types
    # ensuring categorical data makes sense with .describe()

        # Print the information of ride_sharing
        print(ride_sharing.info())

        # Print summary statistics of user_type column
        print(ride_sharing['user_type'].describe())

    # ensuring integers are int and not str

        # Strip duration of minutes
        ride_sharing['duration_trim'] = ride_sharing['duration'].str.strip('minutes')

        # Convert duration to integer
        ride_sharing['duration_time'] = ride_sharing['duration_trim'].astype('int')

        # Write an assert statement making sure of conversion
        assert ride_sharing['duration_time'].dtype == 'int'

In [ ]:
# Constraint 2: Data Inconsistencies 
    # Possibilities: 
    #   lower/upper cases for the same values
    #       solution: lowercase (.str.lower()) or capitalize all values
    #   typos
    #   differences between the same values: "unmarried" or "not married"
    #   trailing spaces: " married" or " married "
    #       solution: .str.strip() without an argument
    #    uniformity
    #   columns related to each other don't match
    #       solution: data integrity/cross field validation
    #                 Ex: age and birthday columns should make sense
    #                 Ex: columns that add together should add to the total column
                        # Store fund columns to sum against
                        fund_columns = ['fund_A', 'fund_B', 'fund_C', 'fund_D']

                        # Find rows where fund_columns row sum == inv_amount
                        inv_equ = ( banking[fund_columns].sum(axis = 1) == banking['inv_amount'] )

                        # Store consistent and inconsistent data
                        consistent_inv = banking[inv_equ]
                        inconsistent_inv = banking[~inv_equ]

                        # Store consistent and inconsistent data
                        print("Number of inconsistent investments: ", inconsistent_inv.shape[0])

In [ ]:
# Constraint 3: Cleaning Text Data
    # solutions: 
        # replacing characters by other characters
        # filtering the dataset based on characters

# Replacing Characters by other Characters
    # Replace "Mr." with empty string ""
    airlines['full_name'] = airlines['full_name'].str.replace("Mr.","")

    # Replace "Ms." with empty string ""
    airlines['full_name'] = airlines['full_name'].str.replace("Ms.","")

    # Assert that full_name has no honorifics
    assert airlines['full_name'].str.contains('Ms.|Mr.').any() == False

# Filtering the dataset based on characters
    # Store length of each row in survey_response column
    resp_length = airlines['survey_response'].str.len()

    # Find rows in airlines where resp_length > 40
    airlines_survey = airlines[resp_length > 40]

    # Assert minimum survey_response length is > 40
    assert airlines_survey['survey_response'].str.len().min() > 40

    # Print new survey_response column
    print(airlines_survey['survey_response'])

In [ ]:
# Constraint 4: Membership (columns must have a predefined set of values)                              
    # the idea is to ensure that a varible that has predefined values only have those values
    # Ex: month variable is supposed to have 1-12 values
    # Ex: a yes or no variable must have only 0-1 values

    # Example
        # Print categories DataFrame
        print(categories[['cleanliness']])
            #       cleanliness
            # 0           Clean
            # 1         Average
            # 2  Somewhat clean
            # 3  Somewhat dirty
            # 4           Dirty

        # Print unique values of survey columns in airlines
        print(airlines['cleanliness'].unique())
            # ['Clean', 'Average', 'Unacceptable', 'Somewhat clean', 'Somewhat dirty', 'Dirty']
        
        # Find the cleanliness category in airlines not in categories
        cat_clean = set(airlines['cleanliness']).difference(categories['cleanliness'])
        print(cat_clean)
            # {'Unacceptable'}
        print(type(cat_clean))
            # <class 'set'>

        
        # Find rows with that category
        cat_clean_rows = airlines['cleanliness'].isin(cat_clean)
        
        # Print rows with consistent categories only
        print(airlines[~cat_clean_rows])

In [ ]:
# Constraint 5: Data Range
    # The idea is to ensure that you only have the data range the business wants
    
        # Example: Categorical Variable with an numerical essense (convert to int and then convert it back to categorical)
            # Convert tire_sizes to integer
            ride_sharing['tire_sizes'] = ride_sharing['tire_sizes'].astype('int')

            # Set all values above 27 to 27
            ride_sharing.loc[ride_sharing['tire_sizes'] > 27, 'tire_sizes'] = 27

            # Reconvert tire_sizes back to categorical
            ride_sharing['tire_sizes'] = ride_sharing['tire_sizes'].astype('category')

            # Print tire size description
            print(ride_sharing['tire_sizes'].describe())
            type(ride_sharing['tire_sizes'])

        # Example: a big has stored bike rides date taken today as if it were taken next year. We need to correct it.
            # Recal that we sometimes need to convert date to a datetime object and then to a date type
            # Convert ride_date to date
            ride_sharing['ride_dt'] = pd.to_datetime(ride_sharing['ride_date']).dt.date

            # Save today's date
            today = dt.date.today()

            # Set all in the future to today's date
            ride_sharing.loc[ride_sharing['ride_dt'] > today, 'ride_dt'] = today

            # Print maximum of ride_dt column
            print(ride_sharing['ride_dt'].max())

In [ ]:
# Constraint 6: Categories Manipulation / Remapping
    # pd.qcut()
    # pd.cut()
    # replace({'original_value':'new_value'})

# Create ranges for categories
label_ranges = [0, 60, 180, np.inf]
label_names = ['short', 'medium', 'long']

# Create wait_type column
airlines['wait_type'] = pd.cut(airlines['wait_min'], bins = label_ranges, 
                                labels = label_names)

# Create mappings and replace
mappings = {'Monday':'weekday', 'Tuesday':'weekday', 'Wednesday': 'weekday', 
            'Thursday': 'weekday', 'Friday': 'weekday', 
            'Saturday': 'weekend', 'Sunday': 'weekend'}

airlines['day_week'] = airlines['day'].replace(mappings)

In [ ]:
# Constraint 7: Date Manipulation
    # Using String Similarity to scale the remapping process
        # Example (establishing the cutoff point for the similarity score)
            # Remember, when comparing a string with an array of strings using process.extract(), the output is a list of tuples
            # where each is formatted like: (closest match, similarity score, index of match)

            # Import process from thefuzz
            from thefuzz import process

            # Store the unique values of cuisine_type in unique_types
            unique_types = restaurants['cuisine_type'].unique()

            # Calculate similarity of 'asian' to all values of unique_types 
            print(process.extract('italian', unique_types, limit = len(unique_types)))
                # [('italian', 100), ('italiann', 93), ('italiano', 93)]
                # 80 seems to be a good cutoff
            
        # Example: Now, let's find matches with similarity scores equal to or higher than 80
            # Create a list of matches, comparing 'italian' with the cuisine_type column
            matches = process.extract('italian', restaurants['cuisine_type'], limit = len(restaurants['cuisine_type']))

            # Inspect the first 5 matches
            print(matches[0:5])
            
            # Iterate through the list of matches to italian
            for match in matches:
                # Check whether the similarity score is greater than or equal to 80
                if match[1] >= 80:
                    # Select all rows where the cuisine_type is spelled this way, and set them to the correct cuisine
                    restaurants.loc[restaurants['cuisine_type'] == match[0]] = 'italian'
            
            # Inspect the final result
            print(restaurants['cuisine_type'].unique())

In [ ]:
# Constraint 8: Date Manipulation
    # Python stores date types columns as Datetime, a special date type.
    # Pandas convert a datetime column as 'object' by default
    # We use parse_dates to specify which columns are of the datetime type and not dtype = {...} as with other columns with other types.

    # Example:
    # Create dict of columns to combine into new datetime column
    datetime_cols = {"Part2Start": ['Part2StartDate', 'Part2StartTime']}

    # Load file, supplying the dict to parse_dates
    survey_data = pd.read_excel("fcc_survey_dts.xlsx",
                                parse_dates = datetime_cols)


    # So far, you've parsed dates that pandas could interpret automatically. But if a date is in a non-standard format, 
    # like 19991231 for December 31, 1999, it can't be parsed at the import stage. 
    # Instead, use pd.to_datetime() to convert strings to dates after import.
    print(survey_data['Part2EndTime'].head())
        # 0    03292016 21:27:25
        # 1    03292016 21:29:10
        # 2    03292016 21:28:21
    # The format above follows: %m%d%Y %H:%M:%S

    # Parse datetimes and assign result back to Part2EndTime
    survey_data["Part2EndTime"] = pd.to_datetime(survey_data["Part2EndTime"], format="%m%d%Y %H:%M:%S")

# Print first few values of Part2EndTime
print(survey_data["Part2EndTime"].head())
    # 0   2016-03-29 21:27:25
    # 1   2016-03-29 21:29:10
    # 2   2016-03-29 21:28:21

# Example:
# We need to convert a date column to a datetime object and then to a date type
        ride_sharing['ride_dt'] = pd.to_datetime(ride_sharing['ride_date']).dt.date

In [ ]:
# Constraint 9: Uniqueness
    # find duplicates 
        #find duplicates based on some column and keep them all in the dataset
        duplicates = ride_sharing.duplicated(subset = 'ride_id', keep = False)
    
    # drop duplicates
        # drop complete duplicates
        ride_dup = ride_sharing.drop_duplicates()

    # clean categories 
        # Print unique values of survey columns in airlines based on a "dimension" table called "categories"
        print('Cleanliness: ', airlines['cleanliness'].unique(), "\n")
        
        # Find the cleanliness category in airlines not in categories
        cat_clean = set(airlines['cleanliness']).difference(categories['cleanliness'])
        print(cat_clean)
        print(type(cat_clean))

        # Find rows with that category
        cat_clean_rows = airlines['cleanliness'].isin(cat_clean)
        print(cat_clean_rows)
        print(type(cat_clean_rows))

        # Print rows with inconsistent category
        print(airlines[cat_clean_rows])

In [2]:
# Constraint 10: Missing Data
    #Missingness Types
        # MCAR - Missing Completely at Random: no systematic relationship between missing data and other values, which makes us
        #        believe that it is due to data entry errors when inputting data
        # MAR - Missing at Random: there is a systematic relationship between missing data and other *observed* values such as CO2 data
        #       being missing for low temperature (a negative correlation - the more missing data the lower the temperature),
        #       which makes us believe that it is a sensor problem
        # MNAR - Missing Not at Random: there is a systematic relationship between missing data and *unobserved* values. 
            # Example:
            #   when it is very hot outside and the thermomether stopped working (so we don't have measures for days with high temp.)
            #   however, notice that we can not infer this from the data because we cannot see what the missing data is (but we found out about it
            #   and it made sense to relate to whatever variables we could not control for - because they were within the error term, hence the 
            #   unobserved idea).
            # Example: 
            #   low values of satisfaction_score are missing because of inherently low satisfaction!

    # Simple solutions: 
        # droping the records with missing values
        # imputing/replacing the values with statistical measures (point estimates)

        # Example
            # Print number of missing values in banking
            print(banking.isna().sum())

            # Visualize missingness matrix
            msno.matrix(banking)
            plt.show()

            # Isolate missing and non missing values of inv_amount
            missing_investors = banking[banking['inv_amount'].isna()]
            investors = banking[~banking['inv_amount'].isna()]

            # Sort banking by age and visualize
            banking_sorted = banking.sort_values(by = 'age')
            msno.matrix(banking_sorted)
            plt.show()
        
        # Example
            # Drop missing values of cust_id
            banking_fullid = banking.dropna(subset = ['cust_id'])

            # Compute estimated acct_amount
            acct_imp = banking_fullid['inv_amount'] * 5

            # Impute missing acct_amount with corresponding acct_imp
            banking_imputed = banking_fullid.fillna({'acct_amount':acct_imp})

            # Print number of missing values for all columns
            print(banking_imputed.isna().sum())


In [ ]:
# Constraint 11: Missing data vs. Boolean
# When Casting a column as boolean (for example, when importing with pandas read_xyz) it is important to check whether
# there is missing values.
# Some columns contain TRUE/FALSE, YES/NO, 0/1 values, which are trying to represent a column of the boolean type.
# When you import data with Pandas, pandas try to guess, but there are issues:
#   1) It guesses wrongly, so even a 0/1 column can be interpreted as float
#   2) When there are missing values in a column, pandas might recode missing values/NAs/NaN as TRUE.

# Load file with Yes as a True value and No as a False value and specify the types of some columns as boolean
# the 'HasDebt' column is a 0/1 column
# 'AttendedBootCampYesNo' is a Yes/No column
survey_subset = pd.read_excel("fcc_survey_yn_data.xlsx",
                              dtype={"HasDebt": bool,
                                    "AttendedBootCampYesNo": bool},
                              true_values=["Yes"],
                              false_values=["No"])

# View the data
print(survey_subset.head())